Function Prediction Model

Training:

In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import pandas as pd

#For DataHandling
class CustomDataset(Dataset):

  def __init__(self, examples, tokenizer, max_length=128):
    self.examples = examples
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, idx):

    example = self.examples[idx]
    text = example["text"]
    label = torch.tensor(example["label"], dtype=torch.long)


    inputs = self.tokenizer(
        text,
        return_tensors="pt",
        max_length=self.max_length,
        padding="max_length",
        truncation=True,
    )

    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "label": label,
    }



excel_file_path = "/content/dataset_multiclass_3.xlsx"

df = pd.read_excel(excel_file_path)

examples = df[["text", "label"]].to_dict("records")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
num_classes = len(df["label"].unique())
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

custom_dataset = CustomDataset(examples, tokenizer)
dataloader = DataLoader(custom_dataset, batch_size=8, shuffle=True)

num_epochs = 5
for epoch in range(num_epochs):
  model.train()
  for batch in dataloader:

    inputs = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["label"]

    #Fpass
    outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    #Bpass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


model.save_pretrained("save/model_multiclass")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 1.7475414276123047
Epoch 2, Loss: 1.6825226545333862
Epoch 3, Loss: 1.2501829862594604
Epoch 4, Loss: 1.183195948600769
Epoch 5, Loss: 1.2933166027069092
Input Prompt: song on
Predicted Function Call: play_music();



Inference:

In [ ]:

#Loading the trained model
model = BertForSequenceClassification.from_pretrained("save/model_multiclass")

model.eval()

input_prompts = ["song on"]

#tokanizing input with dynamic padding
tokenized_inputs = tokenizer(input_prompts, return_tensors="pt", padding=True, truncation=True)


with torch.no_grad():
  logits = model(**tokenized_inputs).logits


probs = torch.nn.functional.softmax(logits, dim=1)

predicted_classes = torch.argmax(probs, dim=1)

class_to_function_mapping = {
    0: "time_date_wolfram();",
    1: "play_music();",
    2: "lights();",
    3: "increase_volume();",
    4: "decrease_volume();",
    5: "increase_brightness();",
    6: "decrease_brightness()"

}


predicted_function_calls = [class_to_function_mapping[class_idx.item()] for class_idx in predicted_classes]

# Print the results
for input_prompt, function_call in zip(input_prompts, predicted_function_calls):

  print(f"Input Prompt: {input_prompt}")
  print(f"Predicted Function Call: {function_call}")
  print()


Dual Model - Funcion Call Prediction with Parameter Prediction: Training

In [19]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import pandas as pd

class FunctionCallModel(BertForSequenceClassification):
    def __init__(self, num_classes):
        super(FunctionCallModel, self).__init__(from_pretrained="bert-base-uncased", num_labels=num_classes)


#Function-call Prediction model
class FunctionCallModel(torch.nn.Module):

  def __init__(self, num_classes):
    super(FunctionCallModel, self).__init__()
    self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

  def forward(self, inputs, attention_mask):

    outputs = self.model(input_ids=inputs, attention_mask=attention_mask, labels=None)
    logits = outputs.logits
    return logits


#Function-parameter Prediction Model
class FunctionParameterModel(torch.nn.Module):
  def __init__(self, num_classes):
    super(FunctionParameterModel, self).__init__()
    self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

  def forward(self, inputs, attention_mask):
    outputs = self.model(input_ids=inputs, attention_mask=attention_mask, labels=None)
    logits = outputs.logits
    return logits



class CustomDataset(Dataset):

  def __init__(self, examples, tokenizer, max_length=128):
    self.examples = examples
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, idx):

    example = self.examples[idx]
    text = str(example["text"])
    label = torch.tensor(example["label"], dtype=torch.long)


    inputs = self.tokenizer(text, return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True,)

    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "label": label,
    }

excel_file_path_function = "/content/dataset_multiclass_3.xlsx"
excel_file_path_parameter = "/content/parameter_dataset_3.xlsx"

df_function = pd.read_excel(excel_file_path_function)
df_parameter = pd.read_excel(excel_file_path_parameter)

examples_function = df_function[["text", "label"]].to_dict("records")
examples_parameter = df_parameter[["text", "label"]].to_dict("records")

tokenizer_function = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer_parameter = BertTokenizer.from_pretrained("bert-base-uncased")

num_classes_function = len(df_function["label"].unique())
num_classes_parameter = len(df_parameter["label"].unique())
model_f = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes_function)
model_p = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes_parameter)
model_function = FunctionCallModel(num_classes_function)
model_parameter = FunctionParameterModel(num_classes_parameter)

optimizer_function = AdamW(model_function.parameters(), lr=2e-5)
optimizer_parameter = AdamW(model_parameter.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

#Set up DataLoaders
custom_dataset_function = CustomDataset(examples_function, tokenizer_function)
dataloader_function = DataLoader(custom_dataset_function, batch_size=8, shuffle=True)

custom_dataset_parameter = CustomDataset(examples_parameter, tokenizer_parameter)
dataloader_parameter = DataLoader(custom_dataset_parameter, batch_size=8, shuffle=True)

#Training for Function-call Prediction Model
print("Training Function Call Prediction Model:")
print(" ")

num_epochs_function = 10
for epoch in range(num_epochs_function):

  model_function.train()
  for batch in dataloader_function:

    inputs = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["label"]


    outputs = model_function(inputs=inputs, attention_mask=attention_mask)
    loss = loss_fn(outputs, labels)

    optimizer_function.zero_grad()
    loss.backward()
    optimizer_function.step()

  print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


model_f.save_pretrained("save/model_multiclass")



"""
torch.save({
    'model_state_dict': model_function.state_dict(),
    'optimizer_state_dict': optimizer_function.state_dict(),
    'loss': loss_fn,
}, "save/model_multiclass.pth")
"""

print("---------------------------------------------------------------")
print("Training Function Parameter Prediction Model:")


num_epochs_parameter = 10
for epoch in range(num_epochs_parameter):

  model_parameter.train()
  for batch in dataloader_parameter:

    inputs = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["label"]


    outputs = model_parameter(inputs=inputs, attention_mask=attention_mask)
    loss = loss_fn(outputs, labels)


    optimizer_parameter.zero_grad()
    loss.backward()
    optimizer_parameter.step()


  print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


model_p.save_pretrained("save/model_parameters")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are

Training Function Call Prediction Model:
 
Epoch 1, Loss: 1.9984384775161743
Epoch 2, Loss: 2.0864670276641846
Epoch 3, Loss: 1.730629801750183
Epoch 4, Loss: 1.559712290763855
Epoch 5, Loss: 1.8500081300735474
Epoch 6, Loss: 1.650430679321289
Epoch 7, Loss: 1.1223037242889404
Epoch 8, Loss: 0.7647712826728821
Epoch 9, Loss: 1.1426070928573608
Epoch 10, Loss: 0.6659547686576843
---------------------------------------------------------------
Training Function Parameter Prediction Model:
Epoch 1, Loss: 1.377889633178711
Epoch 2, Loss: 1.0356718301773071
Epoch 3, Loss: 0.8999375104904175
Epoch 4, Loss: 0.710089921951294
Epoch 5, Loss: 0.5664712190628052
Epoch 6, Loss: 0.4347323477268219
Epoch 7, Loss: 0.2845042645931244
Epoch 8, Loss: 0.1854538470506668
Epoch 9, Loss: 0.6032761931419373
Epoch 10, Loss: 0.12956838309764862


In [ ]:

model_function = BertForSequenceClassification.from_pretrained("save/model_multiclass")
model_parameter = BertForSequenceClassification.from_pretrained("save/model_parameters")


model_function.eval()
model_parameter.eval()


input_prompts = ["on the light"]


tokenized_inputs = tokenizer_function(input_prompts, return_tensors="pt", padding=True, truncation=True)


with torch.no_grad():
  logits_function = model_function(**tokenized_inputs).logits

probs_function = torch.nn.functional.softmax(logits_function, dim=1)

predicted_classes_function = torch.argmax(probs_function, dim=1)

class_to_function_mapping = {
    0: "increase_volume",
    1: "decrease_volume",
    2: "lights_on",
    3: "lights_off",
    4: "increase_brightness",
    5: "decrease_brightness"
}

predicted_function_calls = [class_to_function_mapping[class_idx.item()] for class_idx in predicted_classes_function]

with torch.no_grad():
  logits_parameter = model_parameter(**tokenized_inputs).logits

probs_parameter = torch.nn.functional.softmax(logits_parameter, dim=1)

predicted_classes_parameter = torch.argmax(probs_parameter, dim=1)

class_to_parameter_mapping = {
    0: "10",
    1: "15",
    2: "on",
    3: "off",
}

predicted_parameters = [class_to_parameter_mapping[class_idx.item()] for class_idx in predicted_classes_parameter]

for input_prompt, function_call, parameter in zip(input_prompts, predicted_function_calls, predicted_parameters):
  print(f"Input Prompt: {input_prompt}")
  print(f"Predicted Function Call: {function_call}")
  print(f"Predicted Parameter: {parameter}")
  print()
